In [4]:
!pip install datasets
from datasets import load_dataset, DatasetDict, Dataset

In [4]:
!pip install evaluate

  Using cached evaluate-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached responses-0.18.0-py3-none-any.whl.metadata (29 kB)
Using cached evaluate-0.4.1-py3-none-any.whl (84 kB)
Using cached responses-0.18.0-py3-none-any.whl (38 kB)


In [5]:
! pip install transformers
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

In [6]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

ModuleNotFoundError: No module named 'peft'

In [7]:
model_checkpoint = 'LiyaT3/sentiment-analysis-imdb-distilbert'

id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)
for name, param in model.named_parameters():
    param.requires_grad = False
# num=2
# model.layers[-1].trainable= True
# model.layers[-1].units= num

In [8]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [22]:
for name, param in model.named_parameters():
     if name.startswith("classifier"): # choose whatever you like here
        param.requires_grad = True

In [24]:
from datasets import load_dataset

dataset = load_dataset("sg247/binary-classification")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 8004
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 2000
    })
})


In [10]:
import pandas as pd

df = pd.DataFrame(dataset['train'])
df.dropna(inplace = True)


df['label'] = df['label'].astype(int)
print(df)


                                                  tweet  label
0     Want to say a huge thanks to @WarriorAssaultS ...      1
1     @jaynehh_ you just need a job and get a letter...      1
2             @knhillrocks HA yes, make it quick tho :D      1
3     @shartyboy Thanks for texting me back :)) I'm ...      1
4     Laying out a greetings card range for print to...      1
...                                                 ...    ...
7999  "@aula_jr: @Jude_Mugabi I'm watching it. 2-0 M...      0
8000                                  why so sudden :((      0
8001        @21oclock :((( bout to instant transmission      0
8002  @ButtCupboard my feet were cold!!!! and there ...      0
8003  @Kimwoobin89__ oppa............./hug ur arm/ w...      0

[8000 rows x 2 columns]


In [11]:
d = pd.DataFrame(dataset['test'])
d.dropna(inplace = True)
d['label'] = d['label'].astype(int)


In [12]:
from datasets import Dataset, DatasetDict
train_dict = {
    'tweet': df['tweet'].tolist(),
    'label': df['label'].tolist()
}
train_dataset = Dataset.from_dict(train_dict)
dataset_dict = DatasetDict({'train': train_dataset})
print(dataset_dict['train']['label'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [13]:
from datasets import Dataset, DatasetDict
test_dict = {
    'tweet': d['tweet'].tolist(),
    'label': d['label'].tolist()
}
test_dataset = Dataset.from_dict(train_dict)
dataset_dict = DatasetDict({'test': train_dataset})

# print(dataset_dict['train']['label'][2])
print(dataset_dict['test']['label'][2])
print(dataset_dict)

1
DatasetDict({
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 8000
    })
})


In [25]:
test_dict = {
    'tweet': d['tweet'].tolist(),
    'label': d['label'].tolist()
}

train_dict = {
    'tweet': df['tweet'].tolist(),
    'label': df['label'].tolist()
}



import datasets
train_dataset = datasets.Dataset.from_dict(train_dict)
test_dataset = datasets.Dataset.from_dict(test_dict)
my_dataset_dict = datasets.DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})
print(my_dataset_dict)
print(my_dataset_dict['test']['label'][2])
print(my_dataset_dict['train']['label'][2])

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 2000
    })
})
1
1


In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

def tokenize_function(examples):
    tweets = examples["tweet"]
    preprocess_t= [tweet if isinstance(tweet , str) else str(tweet) for tweet in tweets]
    return tokenizer(preprocess_t , truncation= True , padding= "max_length")

tokenized_dataset = my_dataset_dict.map(tokenize_function, batched=True)
tokenized_dataset

Map: 100%|██████████| 2000/2000 [00:00<00:00, 5347.84 examples/s]


DatasetDict({
    train: Dataset({
        features: ['tweet', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['tweet', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [27]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [28]:
lr = 0.01
batch_size = 32
num_epochs = 1

In [29]:
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [30]:
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    eval_dataset = tokenized_dataset['train'],
    compute_metrics=compute_metrics, 
)

In [32]:
trainer.train()





























































































































































































































































                                                 

  0%|          | 0/250 [03:18<?, ?it/s]          



{'eval_loss': 0.16478970646858215, 'eval_accuracy': 0.9525, 'eval_runtime': 59.7988, 'eval_samples_per_second': 133.782, 'eval_steps_per_second': 4.181, 'epoch': 1.0}



100%|██████████| 250/250 [02:04<00:00,  2.01it/s]

{'train_runtime': 124.3835, 'train_samples_per_second': 64.317, 'train_steps_per_second': 2.01, 'train_loss': 0.29259954833984375, 'epoch': 1.0}


NameError: name 'torch' is not defined